# Παίζοντας Atari με βαθιά ενισχυτική μάθηση

## Σετάρισμα

In [ ]:
!apt install apt-utils psmisc -y
!killall -9 ngrok
!rm -rf *ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir -p ngrok-dir
!mv ngrok ngrok-dir/
# Στην επόμενη γραμμή βάλτε το δικό σας authentication token από το ngrok
!ngrok-dir/ngrok authtoken 1QFJyhgjhEYDERjvBEmb8wMBLrO_3JN3bTDmy7sJWAmKoby8p

LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('ngrok-dir/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("Wait 10-15 seconds and then click the URL above to open TensorBoard")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
apt-utils is already the newest version (1.6.12ubuntu0.2).
psmisc is already the newest version (23.1-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
ngrok: no process found
--2021-03-11 14:53:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.164.22.162, 54.145.36.98, 3.212.138.198, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.164.22.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.7MB/s    in 0.7s    

2021-03-11 14:53:05 (18.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ng

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

video_folder = './videos'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 784 kB in 1s (584 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.8_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.8) ...
Setting up xvfb (2:1.19.6-1ubuntu4.8) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


### Εγκατάσταση βιβλιοθήκης και gym

In [ ]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3
!pip install sb3-contrib

     |████████████████████████████████| 1.5MB 8.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 152 kB 7.9 MB/s 
     |████████████████████████████████| 2.8 MB 13.8 MB/s 
     |████████████████████████████████| 2.1 MB 56.0 MB/s 
     |████████████████████████████████| 104 kB 34.0 MB/s 
     |████████████████████████████████| 1.1 MB 68.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 41 kB 418 kB/s 
     |████████████████████████████████| 66 kB 6.1 MB/s 
     |████████████████████████████████| 719 kB 54.9 MB/s 
     |████████████████████████████████| 743 kB 74.5 MB/s 
     |████████████████████████████████| 280 kB 68.7 MB/s 
     |████████████████████████████████| 242 kB 67.5 MB/s 
     |███████████

In [ ]:
from stable_baselines3 import DQN, A2C, HER, PPO
from sb3_contrib import QRDQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import datetime # For filenames while logging
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.bit_flipping_env import BitFlippingEnv

## Ορισμός παιχνιδιού

Παρακάτω χειροκίνητα (με uncomment) επιλέγουμε το περιβάλλον του παιχνιδιού.  

In [ ]:
atari_env_name='Atlantis-v0'
# atari_env_name='Atlantis-v4'
# atari_env_name='AtlantisDeterministic-v0'
# atari_env_name='AtlantisDeterministic-v4'
# atari_env_name='AtlantisNoFrameskip-v0'
# atari_env_name='AtlantisNoFrameskip-v4'

## Δημιουργία περιβάλλοντος

Φτιάχνουμε $2$ περιβάλλοντα, ένα με multi-processing και ένα χωρίς, ώστε να έχουμε συμβατότητα με τους Reinforcement learning αλγορίθμους της SB$3$.

In [ ]:
# Με τις συναρτήσεις που ακολουθούν κάνουμε την ίδια προεπεξεργασία με την DeepMind

# Here we are also multi-worker training (n_envs=4 => 4 environments).
# The model must support Multi Processing. 
# Tα DQN,  HER δεν επιτρέπουν multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
env_multi = make_atari_env(atari_env_name, n_envs=4, seed=0)

# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)
env_multi = VecFrameStack(env_multi, n_stack=4)

# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)

# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

## Εκπαίδευση

#### DQN

In [ ]:
model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=100000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model.save("dqn_atlantis")

#### A2C

In [ ]:
model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env_multi, verbose=1, tensorboard_log=model_log)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=10000
a2c_model.learn(total_timesteps=max_steps)

Logging to tb_log/a2c-MlpPolicy-20210307-232108/A2C_1
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 668       |
|    ep_rew_mean        | 4.11e+03  |
| time/                 |           |
|    fps                | 219       |
|    iterations         | 100       |
|    time_elapsed       | 9         |
|    total_timesteps    | 2000      |
| train/                |           |
|    entropy_loss       | -1.18     |
|    explained_variance | -5.96e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.0355   |
|    value_loss         | 0.000814  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 700       |
|    ep_rew_mean        | 4.47e+03  |
| time/                 |           |
|    fps                | 223       |
|    iterations         | 200       |
|    time_elapsed       | 17      

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 2020.0 (+/-40.0)


In [ ]:
a2c_model.save("a2c_atlantis")

#### PPO

In [ ]:
model_name='ppo-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

ppo_model = PPO('MlpPolicy', env_multi, verbose=1, tensorboard_log=model_log)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=100000
ppo_model.learn(total_timesteps=max_steps)

Logging to tb_log/ppo-MlpPolicy-20210305-203417/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 709      |
|    ep_rew_mean     | 4.34e+03 |
| time/              |          |
|    fps             | 255      |
|    iterations      | 1        |
|    time_elapsed    | 32       |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 736         |
|    ep_rew_mean          | 4.96e+03    |
| time/                   |             |
|    fps                  | 197         |
|    iterations           | 2           |
|    time_elapsed         | 82          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.021003269 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_var

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 5100.0 (+/-1566.5248162732692)


In [ ]:
ppo_model.save("ppo_atlantis")

#### QR-DQN

In [ ]:
model_name='qrdqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

qrdqn_model = QRDQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=100000
qrdqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(qrdqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
qrdqn_model.save("qrdqn_atlantis")

## Βελτιστοποίηση

### A2C

In [ ]:
model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env_multi, verbose=1, tensorboard_log=model_log)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=300000
a2c_model.learn(total_timesteps=max_steps)

Logging to tb_log/a2c-MlpPolicy-20210308-184854/A2C_1
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 637       |
|    ep_rew_mean        | 4.23e+03  |
| time/                 |           |
|    fps                | 242       |
|    iterations         | 100       |
|    time_elapsed       | 8         |
|    total_timesteps    | 2000      |
| train/                |           |
|    entropy_loss       | -1.31     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 0.266     |
|    value_loss         | 0.229     |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 684      |
|    ep_rew_mean        | 5.13e+03 |
| time/                 |          |
|    fps                | 245      |
|    iterations         | 200      |
|    time_elapsed       | 16       |
|   

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 2030.0 (+/-45.8257569495584)


In [ ]:
model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env_multi, verbose=1, tensorboard_log=model_log, gae_lambda=0.5)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=200000
a2c_model.learn(total_timesteps=max_steps)

Logging to tb_log/a2c-MlpPolicy-20210308-183234/A2C_1
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 557       |
|    ep_rew_mean        | 3.09e+03  |
| time/                 |           |
|    fps                | 236       |
|    iterations         | 100       |
|    time_elapsed       | 8         |
|    total_timesteps    | 2000      |
| train/                |           |
|    entropy_loss       | -0.695    |
|    explained_variance | -3.93e-06 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 0.134     |
|    value_loss         | 0.0648    |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 548      |
|    ep_rew_mean        | 2.96e+03 |
| time/                 |          |
|    fps                | 237      |
|    iterations         | 200      |
|    time_elapsed       | 16       |
|   

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 2050.0 (+/-50.0)


In [ ]:
a2c_model.save("a2c_atlantis")

### PPO

In [ ]:
model_name='ppo-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

ppo_model = PPO('MlpPolicy', env_multi, verbose=1, tensorboard_log=model_log, learning_rate=0.00003, gamma=0.9)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=100000
ppo_model.learn(total_timesteps=max_steps)

Logging to tb_log/ppo-MlpPolicy-20210307-225047/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 695      |
|    ep_rew_mean     | 4.39e+03 |
| time/              |          |
|    fps             | 270      |
|    iterations      | 1        |
|    time_elapsed    | 30       |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 714         |
|    ep_rew_mean          | 4.89e+03    |
| time/                   |             |
|    fps                  | 200         |
|    iterations           | 2           |
|    time_elapsed         | 81          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.014404379 |
|    clip_fraction        | 0.0765      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_var

In [ ]:
# 0.00003 0.9
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 1702580.0 (+/-1640608.892332356)


In [ ]:
ppo_model.save("ppo_atlantis_00003_09")

## Φόρτωση εκπαιδευμένου μοντέλου


In [ ]:
!wget --no-check-certificate 'https://www.dropbox.com/s/0ur34qf7czhm95o/ppo_atlantis_00003_09.zip?dl=1' -O ppo_atlantis.zip

--2021-03-11 16:04:17--  https://www.dropbox.com/s/0ur34qf7czhm95o/ppo_atlantis_00003_09.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/0ur34qf7czhm95o/ppo_atlantis_00003_09.zip [following]
--2021-03-11 16:04:17--  https://www.dropbox.com/s/dl/0ur34qf7czhm95o/ppo_atlantis_00003_09.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc005235ea38d90aa19ec084c6f1.dl.dropboxusercontent.com/cd/0/get/BKeBobkMDkkEl0UZuEOSwn9TwHfQJDDgw9iGdOd0dDl1dF7yDPIDXjgH3zPuYL1Q1A1yz9ZJNC8ss2xXTFzLYBdGLMU6pSSBmrzmLQOYyyHZpOQsNMxdk5e77CSDdHQvJ6qbZc7syXKRjE-ZQXUp1840/file?dl=1# [following]
--2021-03-11 16:04:17--  https://uc005235ea38d90aa19ec084c6f1.dl.dropboxusercontent.com/cd/0/get/BKeBobkMDkkEl0UZuEOSwn9TwHfQJDDgw9iGdOd0dDl1dF7yD

## Συνέχιση της εκπαίδευσης

Την διαδικασία συνέχισης εκπαίδευσης εφαρμόσαμε μόνο στο μοντέλο PPO, ωστόσο παραθέτουμε και τον κώδικα που αφορά το μοντέλο A2C.

In [ ]:
ppo_model = PPO.load("ppo_atlantis", verbose=1)

In [ ]:
# Προσοχή, το n_envs πρέπει να είναι πάντα το ίδιο (αυτό με το οποίο έγινε η αρχική εκπαίδευση του μοντέλου)
new_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
new_env_multi = make_atari_env(atari_env_name, n_envs=4, seed=0)

# Frame-stacking with 4 frames
new_env = VecFrameStack(new_env, n_stack=4)
new_env_multi = VecFrameStack(new_env_multi, n_stack=4)

# Το μοντέλο δεν έχει μαζί του το περιβάλλον, πρέπει να του το αναθέσουμε ξανά.
# dqn_model.set_env(new_env)
# a2c_model.set_env(new_env_multi)
# her_model.set_env(new_env)
ppo_model.set_env(new_env_multi)
# qrdqn_model.set_env(new_env_multi)

Wrapping the env in a VecTransposeImage.


### A2C

In [ ]:
max_steps=100000
a2c_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
a2c_model.save("a2c_atlantis")

### PPO

In [ ]:
max_steps=200000
ppo_model.learn(total_timesteps=max_steps)

Logging to tb_log/ppo-MlpPolicy-20210307-125515/PPO_5
-----------------------------
| time/              |      |
|    fps             | 296  |
|    iterations      | 1    |
|    time_elapsed    | 27   |
|    total_timesteps | 8192 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 219         |
|    iterations           | 2           |
|    time_elapsed         | 74          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.021519639 |
|    clip_fraction        | 0.0809      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.205      |
|    explained_variance   | 0.744       |
|    learning_rate        | 3e-05       |
|    loss                 | 0.0246      |
|    n_updates            | 2120        |
|    policy_gradient_loss | 0.00438     |
|    value_loss           | 0.0728      |
----------------------

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 1630940.0 (+/-1585017.561543089)


In [ ]:
ppo_model.save("ppo_atlantis_00003_09")

## Καταγραφή video του actual gameplay του πράκτορα

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
record_video(test_env, ppo_model, video_length=20000, prefix='ppo_atlantis')

Saving video to  /content/videos/ppo_atlantis-step-0-to-step-20000.mp4


In [ ]:
show_videos(video_path = video_folder, prefix='ppo')

Output hidden; open in https://colab.research.google.com to view.